In [1]:
# Third test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [4]:
# Global Setup
# Creating temp local repos to test on

TMP=/tmp/temp-git-test


git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"

# Move to second local to push some changes
command cd "$TMP" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt
git commit -m "first commit on second local"
git push


Cloning into '/tmp/temp-git-test'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 286 (delta 37), reused 13 (delta 0), pack-reused 236
Receiving objects: 100% (286/286), 24.00 KiB | 8.00 MiB/s, done.
Resolving deltas: 100% (171/171), done.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
[main 5e117e0] first commit on second local
 1 file changed, 1 insertion(+)
 create mode 100644 hello.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 6 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 310 bytes | 310.00 KiB/s, done.
Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   8762d95..5e117e0  main -> main


In [5]:
# git-diff-plus tests
cat hello.txt > bye.txt
echo "differ" >> hello.txt
diffs=$(git-diff-plus hello.txt bye.txt | grep "^\+\w" | wc -l)


In [6]:
# Assert 
# there should only be one line of difference
[ $diffs -eq 1 ]; echo $?;

0


In [9]:
# git-diff-list tests
git-diff-list 
changed=$(git-diff-list | wc -l)

hello.txt


In [10]:
# Assert 
# there should be one file changed
[ $changed -eq 1 ]; echo $?;

0


In [2]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null
git rm -f *.txt > /dev/null
git add . > /dev/null
git commit -m "cleanup test files"
git push

[main 8762d95] cleanup test files
 1 file changed, 2 deletions(-)
 delete mode 100644 hello.txt
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 6 threads
Compressing objects: 100% (1/1), done.
Writing objects: 100% (2/2), 243 bytes | 243.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   029da95..8762d95  main -> main


In [3]:
# delete test local repos

command cd "/tmp"
rm -rf temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous
